In [1]:
from clasificadores_lineales import *
import numpy as np

In [2]:
# Generamos un conjunto de datos linealmente separables, 
X1,Y1=genera_conjunto_de_datos_l_s(4,8,400)

# Lo partimos en dos trozos:
X1e,Y1e=X1[:300],Y1[:300]

X1t,Y1t=X1[300:],Y1[300:]

# Creamos el clasificador (perceptrón umbral en este caso): 
clas_pb1=Clasificador_Perceptron([0,1], normalizacion = True)

# Lo entrenamos con elprimero de los conjuntos de datos:
clas_pb1.entrena(X1e,Y1e,100,rate_decay=True,rate=0.001)

# Clasificamos un ejemplo del otro conjunto, y lo comparamos con su clase real:
clas_pb1.clasifica(X1t[0]),Y1t[0]

sum(clas_pb1.clasifica(x) == y for x,y in zip(X1t,Y1t))/len(Y1t)

0.96999999999999997

In [3]:
#Igual pero con datos NO linealmente separables
X2,Y2 = genera_conjunto_de_datos_n_l_s(4,8,400,0.1)

X2e,Y2e=X2[:300],Y2[:300]

X2t,Y2t=X2[300:],Y2[300:]

clas_pb2=Clasificador_Perceptron([0,1])

clas_pb2.entrena(X2e,Y2e,100,rate_decay=True,rate=0.001)

clas_pb2.clasifica(X2t[0]),Y2t[0]

sum(clas_pb2.clasifica(x) == y for x,y in zip(X2t,Y2t))/len(Y2t)

0.75

In [4]:
a = np.array([[9987.7,54.8,3],[4,45,20]])
media = np.mean(a, axis = 0)
desviacion = np.std(a, axis = 0)
a = (a - media) / desviacion
print(media, desviacion)
a

[ 4995.85    49.9     11.5 ] [  4.99185000e+03   4.90000000e+00   8.50000000e+00]


array([[ 1.,  1., -1.],
       [-1., -1.,  1.]])